In [1]:
import os
import json
import gzip
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')
from tqdm.autonotebook import tqdm
from atomate2.lobster.schemas import LobsterTaskDocument, CondensedBondingAnalysis
from pydantic import create_model
from pymatgen.io.lobster import Charge,MadelungEnergies

In [2]:
parent=os.getcwd()
os.chdir(parent)
os.chdir('Results/')

In [3]:
mpids= [f for f in os.listdir() if not f.startswith('t') and not f.startswith('.') and not f.startswith('__')
            and os.path.isdir(f)]
mats= list(set([ids.split('_')[0] for ids in mpids]))
mats.sort()

In [4]:
def get_lobster_lightweight_json(mpid, 
    json_save_dir='/hpc-user/AG-JGeorge/anaik/Phonon_dataset_LSO/Lightweight_lobster_jsons/'):
    
    directory = mpid
    
    which_bonds=['cation-anion','all']
    with gzip.open(os.path.join(json_save_dir, "{}.json.gz".format(mpid)), 'wt',encoding='UTF-8') as f:
            f.write('[')
            for which_bond in which_bonds:
                    try:
                        (lobsterpy_data,
                         lobsterpy_text,
                         sb_icobi,
                         sb_icohp,
                         sb_icoop) = CondensedBondingAnalysis.from_directory(dir_name=mpid, which_bonds=which_bond,
                                                                       save_cohp_plots=False)
                        #data = CondensedBondingAnalysis.from_directory(dir_name=mpid, which_bonds=which_bond,
                        #                                               save_cohp_plots=False)


                        bond_type = which_bond.replace('-','_')
                        dict_data= {bond_type+'_bonds':{
                        'lobsterpy_data': lobsterpy_data.dict(),
                        'lobsterpy_text': ["".join(lobsterpy_text.text)],
                        'sb_icobi': sb_icobi.dict(),
                        'sb_icohp': sb_icohp.dict(),
                        'sb_icoop': sb_icoop.dict(),
                        }}

                        for item in dict_data[bond_type+'_bonds']['lobsterpy_data']['cohp_plot_data'].items():
                            key, value = item
                            # check if item has a `as_dict` method (i.e. it is a pymatgen object)
                            if hasattr(value, 'as_dict'):
                                dict_data[bond_type+'_bonds']['lobsterpy_data']['cohp_plot_data'][key]=value.as_dict()            
                        json.dump(dict_data, f)

                        if which_bond != which_bonds[-1]: 
                            f.write(',') #add comma seperator between two dicts
                            
                    except AttributeError:
                        dict_data={bond_type+'_bonds':{}}
                        json.dump(dict_data, f)
                        
                        if which_bond != which_bonds[-1]: 
                            f.write(',') #add comma seperator between two dicts
                            
            f.write(',')        
            madelung_energies_path = os.path.join(mpid, "MadelungEnergies.lobster.gz")
            charge_path = os.path.join(mpid,"CHARGE.lobster.gz")
            madelung_obj = MadelungEnergies(filename=madelung_energies_path)

            madelung_energies = {'madelung_energies':{
                "Mulliken": madelung_obj.madelungenergies_Mulliken,
                "Loewdin": madelung_obj.madelungenergies_Loewdin,
                "Ewald_splitting": madelung_obj.ewald_splitting,
            }}
            json.dump(madelung_energies, f)
            f.write(',')
            charge = Charge(charge_path)
            charges = {"charges":{"Mulliken": charge.Mulliken, "Loewdin": charge.Loewdin}}
            json.dump(charges, f)
                    #f.write(',')
                        
                    #except IndexError:
                    #    print(mpid)
                    #    pass
                    #except TypeError:
                    #    print('Type Error for this:',mpid)
                    #    pass
            f.write(']')
    return mpid+' Done'

In [ ]:
# Key Error : 
#mp-10159, mp-10378, mp-10289, mp-15724, mp-15988, mp-984761,mp-961713,mp-961682,mp-961659,mp-866141,
#mp-867339,mp-7173,mp-697030, mp-697038,mp-643002,mp-581024,mp-569779,mp-5077,mp-35152,mp-20450,mp-2894
#mp-24292,mp-23222,mp-2074,mp-15988,mp-15724

In [ ]:
get_lobster_lightweight_json(mpid='mp-149')

In [ ]:
# Attribute error : mp-149, mp-165, mp-2695, mp-66

In [5]:
with mp.Pool(processes=12,maxtasksperchild=1) as pool:
    results = tqdm(
        pool.imap_unordered(get_lobster_lightweight_json, mats, chunksize=1),
        total=len(mats),
    )
    row=[]
    for result in results:
        row.append(result)

  0%|          | 0/1520 [00:00<?, ?it/s]

In [ ]:
# Weak bonds than cutoff description fails
#mp-632319
#mp-989642

In [ ]:
from pymatgen.electronic_structure.cohp import Cohp
from pymatgen.electronic_structure.plotter import CohpPlotter

In [ ]:
def read_lobster_lightweight_json(filename: str):
    """
    This function loads the json.gz files and returns a dictionary with lobsterpy summmarized bonding analysis.
    
    Args:
        filename (str) : name of file
    Returns:
        Returns a dictionary with lobstepy summmarized bonding analysis output.
    """
    with gzip.open(filename, 'rb') as f:
        data = json.loads(f.read().decode('utf-8'))
        
    lobsterpy_data = {}
    for item in data:
        lobsterpy_data.update(item)
   
    return lobsterpy_data

In [ ]:
lobsterpy_summary = read_lobster_lightweight_json(filename='../Lightweight_lobster_jsons/mp-682.json.gz')

In [ ]:
lobsterpy_summary['all_bonds']['lobsterpy_text']

In [ ]:
lobsterpy_summary

In [ ]:
lobsterpy_summary['all_bonds']['sb_icohp']

In [ ]:
lobsterpy_summary['all_bonds']['sb_icobi']

In [ ]:
lobsterpy_summary['all_bonds']['sb_icoop']

In [ ]:
for number, (key, cohp) in enumerate(lobsterpy_summary['all']["lobsterpy_data"]["cohp_plot_data"].items()):
    plotter = CohpPlotter()
    cohp = Cohp.from_dict(cohp)
    plotter.add_cohp(key, cohp)
    plotter.show()